In [13]:
!pip install python-docx

In [14]:
from docx import Document
import os

# 資料夾路徑
folder_path = r"C:\Users\PC01\Desktop\113-2 Python\Untitled Folder 3"

# 尋找所有以 student 開頭的 docx 檔案
for filename in os.listdir(folder_path):
    if filename.endswith(".docx") and filename.startswith("student"):
        file_path = os.path.join(folder_path, filename)
        print(f"📄 正在讀取：{filename}")

        doc = Document(file_path)
        for para in doc.paragraphs:
            print(para.text)
        print("=" * 40)

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'C:\\Users\\PC01\\Desktop\\113-2 Python\\Untitled Folder 3'

In [48]:
from docx import Document
import os

# 資料夾路徑
folder_path = r"C:\Users\user\Desktop\C111133118朱峻佑\Final"

# 尋找所有以 student 開頭的 docx 檔案
for filename in os.listdir(folder_path):
    if filename.endswith(".docx") and filename.startswith("student"):
        file_path = os.path.join(folder_path, filename)
        print(f"📄 正在讀取：{filename}")

        doc = Document(file_path)
        for para in doc.paragraphs:
            print(para.text)
        print("=" * 40)

📄 正在讀取：student.docx
Unit 5 文法
一、填充題（Choose the correct answer）
1. I felt the ground _____ during the earthquake.(B)
(A) shake
(B) shaking
(C) shook
(D) shakes
2. Not only the players but also the coach _____ nervous before the game.(C)
(A) were
(B) is
(C) are
(D) was
3. Although it’s late, they still _____ video games.(A)
(A) play
(B) played
(C) are playing
(D) plays
4. I heard the baby _____ in the room last night.(A)
(A) crying
(B) cries
(C) to cry
(D) cried
5. Not only Amy but also her classmates’ _____ excited about the school trip.(D)
(A) is
(B) are
(C) was
(D) be

6. Although she was scared, she _____ the haunted house. (C)
(A) enters
(B) entering
(C) entered
(D) enter
7. We saw the teacher _____ into the classroom.(B)
(A) come
(B) coming
(C) came
(D) comes
8. Not only my brother but also I _____ interested in science.(C)
(A) am
(B) is
(C) are
(D) be
9. Although he didn't study, he _____ the test.(C)
(A) passes
(B) passed
(C) is passing
(D) pass
10. I watched the stars _____ in t

In [28]:
from docx import Document
import re
import os

def extract_lines(path):
    doc = Document(path)
    return [para.text.strip() for para in doc.paragraphs if para.text.strip()]

def extract_answers_from_question(lines):
    answers = []
    pattern = re.compile(r"\(([A-Da-d])\)")
    for line in lines:
        match = pattern.search(line)
        if match:
            answers.append(match.group(1).upper())
    return answers

def extract_choices(lines):
    choices = []
    for line in lines:
        parts = line.split()
        if len(parts) >= 2:
            choices.append(parts[-1].upper())
    return choices

# === 資料夾路徑 ===
folder = r"C:\Users\PC01\Desktop\113-2 Python\Untitled Folder 3"

# === 抓出所有答案檔 ===
answer_files = [f for f in os.listdir(folder) if f.startswith("answer") and f.endswith(".docx")]

# === 批改每一份學生考卷 ===
for filename in os.listdir(folder):
    if filename.startswith("student") and filename.endswith(".docx"):
        student_path = os.path.join(folder, filename)
        student_lines = extract_lines(student_path)
        student_answers = extract_answers_from_question(student_lines)

        best_score = -1
        best_answer_file = ""
        best_detail = []

        for answer_file in answer_files:
            answer_path = os.path.join(folder, answer_file)
            correct_answers = extract_choices(extract_lines(answer_path))

            score = 0
            detail = []

            for i, (correct, student) in enumerate(zip(correct_answers, student_answers), start=1):
                if correct == student:
                    score += 10
                    result = "✅"
                else:
                    result = "❌"
                detail.append(f"{result} 第{i}題：學生答 {student}，正確答案 {correct}")
            if score > best_score:
                best_score = score
                best_answer_file = answer_file
                best_detail = detail

        # === 輸出結果 ===
        print(f"📘 批改檔案：{filename}")
        print(f"🔍 最佳答案檔：{best_answer_file}")
        for d in best_detail:
            print(d)
        print(f"🎯 總分：{best_score} / {len(best_detail) * 10}")
        print("=" * 40)



FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'C:\\Users\\PC01\\Desktop\\113-2 Python\\Untitled Folder 3'

In [4]:
!pip install line-bot-sdk openai python-docx schedule

In [7]:
from docx import Document
import re
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

def extract_lines(path):
    doc = Document(path)
    return [para.text.strip() for para in doc.paragraphs if para.text.strip()]

def extract_answers_from_question(lines):
    answers = []
    pattern = re.compile(r"\(([A-Da-d])\)")
    for line in lines:
        match = pattern.search(line)
        if match:
            answers.append(match.group(1).upper())
    return answers

def extract_choices(lines):
    choices = []
    pattern = re.compile(r"\(([A-Da-d])\)|\b([A-Da-d])\b")
    for line in lines:
        match = pattern.search(line)
        if match:
            choice = match.group(1) or match.group(2)
            choices.append(choice.upper())
    return choices

folder = r"C:\Users\user\Desktop\C111133118朱峻佑\Final"

for filename in os.listdir(folder):
    if filename.startswith("student") and filename.endswith(".docx"):
        student_path = os.path.join(folder, filename)
        student_lines = extract_lines(student_path)
        student_answers = extract_answers_from_question(student_lines)

        suffix = filename[7:]
        answer_file = f"answer{suffix}"
        answer_path = os.path.join(folder, answer_file)

        if not os.path.exists(answer_path):
            print(f"⚠️ 找不到對應的答案檔：{answer_file}")
            continue

        correct_answers = extract_choices(extract_lines(answer_path))

        wrong_text = ""
        score = 0
        for i, (correct, student) in enumerate(zip(correct_answers, student_answers), start=1):
            if correct != student:
                question_line = student_lines[i - 1]
                wrong_text += f"第{i}題：{question_line}\n學生答案：{student}，正確答案：{correct}\n\n"
            else:
                score += 10

        print(f"🎯 {filename} 總分：{score} / {len(correct_answers) * 10}")
        
        if wrong_text:
            response = client.chat.completions.create(
                model="gpt-4",  # 或 gpt-3.5-turbo
                messages=[
                    {
                        "role": "system",
                        "content": "你是一位英文老師，請將以下學生答錯的題目整理成教學用的錯題解析。每題請加上文法說明與一個類似的練習題。"
                    },
                    {
                        "role": "user",
                        "content": wrong_text
                    }
                ]
            )
            print("📘 錯題彙整結果：\n")
            print(response.choices[0].message.content)


🎯 student.docx 總分：20 / 100
📘 錯題彙整結果：

第1題：Unit 5 文法
正確答案：A
文法說明：這一題考查的文法點沒有明確給出，需要知道學生錯在哪里才能做適當的解釋。
練習題：?

第2題：一、填充題（Choose the correct answer）
正確答案：D
文法說明：同上，需要明確的語境與文法點才能給予解釋。
練習題：?

第3題：
正確答案：C
文法說明：考查的是感官動詞的用法。感官動詞後面常用一個賓語加一個現在分詞形成的結構來表示正在進行的動作。
練習題：I saw him ___ (run) in the park.

第5題：
正確答案：B
文法說明：這裡考查的是現在進行時的用法。從句子中的"Although it's late"這一句話來看，應該採用現在進行時來表示動作正在進行。
練習題：Despite the rain, they still ___ (play) soccer.

第7題：
正確答案：A
文法說明：“Not only ...but also...”這種結構連接兩個主詞時，動詞的數應該和最近的主詞保持一致。
練習題：“Not only his parents but also his sister ___(be) at the concert.”

第8題：
正確答案：A
文法說明：此處應該使用一般過去時，因為雖然她害怕，但她還是進了鬼屋，事件已經發生，為過去事件。
練習題：“Although he was tired, he ___ (finish) his homework.”

第9題：
正確答案：B
文法說明：這裡考查的是感官動詞的用法，我們看到老師進了教室，這裡要用動詞的現在分詞形式來表示動作正在進行。
練習題：They felt the temperature ___ (rise). 

第10題：
正確答案：D
文法說明：“Not only ...but also...”這種結構連接兩個主詞時，動詞的數應該和最近的主詞保持一致。
練習題：“Not only my parents but also I ___(be) in Paris.”
🎯 student1.docx 總分：10 / 100
📘 錯題彙整結果：

第1題解析：
此為填充題，原始題目已遺失，因此無法進行詳細說明。但是

In [13]:
import os
import re
import random
from docx import Document
from openai import OpenAI
from linebot.v3.messaging import MessagingApi, Configuration, ApiClient
from linebot.v3.messaging.models import PushMessageRequest, TextMessage
from dotenv import load_dotenv

# ✅ 載入 .env 中的環境變數
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
line_channel_access_token = os.getenv("LINE_CHANNEL_ACCESS_TOKEN")
user_id = os.getenv("USER_ID")

# ✅ 初始化 API
client = OpenAI(api_key=openai_api_key)
line_config = Configuration(access_token=line_channel_access_token)
api_client = ApiClient(line_config)
line_bot_api = MessagingApi(api_client)

# ✅ 基礎函式
def extract_lines(path):
    doc = Document(path)
    return [para.text.strip() for para in doc.paragraphs if para.text.strip()]

def extract_answers_from_question(lines):
    answers = []
    pattern = re.compile(r"\(([A-Da-d])\)|答案[:：]?\s*([A-Da-d])|正解[:：]?\s*([A-Da-d])")
    for line in lines:
        match = pattern.search(line)
        if match:
            for group in match.groups():
                if group:
                    answers.append(group.upper())
                    break
    return answers

def extract_choices(lines):
    pattern = re.compile(r"\(([A-Da-d])\)|\b([A-Da-d])\b")
    choices = []
    for line in lines:
        match = pattern.search(line)
        if match:
            choice = match.group(1) or match.group(2)
            choices.append(choice.upper())
    return choices

# ✅ 錯題處理
folder = r"C:\Users\user\Desktop\C111133118朱峻佑\Final"
wrong_questions = []

for filename in os.listdir(folder):
    if filename.startswith("student") and filename.endswith(".docx"):
        student_path = os.path.join(folder, filename)
        student_lines = extract_lines(student_path)
        student_answers = extract_answers_from_question(student_lines)

        suffix = filename[7:]  # 如 1.docx
        answer_file = f"answer{suffix}"
        answer_path = os.path.join(folder, answer_file)

        if not os.path.exists(answer_path):
            print(f"⚠️ 找不到對應的答案檔：{answer_file}")
            continue

        correct_answers = extract_choices(extract_lines(answer_path))
        print(f"📄 處理檔案：{filename}")
        print(f"👉 答案數量：學生 {len(student_answers)}，標準 {len(correct_answers)}")

        if len(correct_answers) != len(student_answers):
            print("❗ 答案數不一致，跳過此份")
            continue

        score = 0
        for i, (correct, student) in enumerate(zip(correct_answers, student_answers), start=1):
            if correct != student:
                wrong_questions.append({
                    "題號": i,
                    "題目": student_lines[i - 1],
                    "學生答案": student,
                    "正確答案": correct
                })
            else:
                score += 10

        print(f"🎯 {filename} 總分：{score} / {len(correct_answers) * 10}")

# ✅ GPT 出題 + LINE 推播
if wrong_questions:
    q = random.choice(wrong_questions)

    prompt = (
        f"請依照這個英文選擇題的風格出一題相似的題目：\n\n"
        f"{q['題目']}\n\n"
        f"題目必須是英文文法或用法的選擇題，包含一題題幹與四個選項 (A)-(D)，並提供正確答案。\n"
        f"格式如下：\n題目：...\n(A) ...\n(B) ...\n(C) ...\n(D) ...\n正確答案：X"
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "你是一位英文老師，擅長出英文選擇題。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
    )

    gpt_output = response.choices[0].message.content.strip()

    # 擷取答案
    match = re.search(r"正確答案[:：]?\s*\(?([A-D])\)?", gpt_output)
    correct_answer = match.group(1) if match else ""

    # 拿掉答案那行
    lines = gpt_output.splitlines()
    question_lines = [line for line in lines if not re.search(r"正確答案[:：]?", line)]
    question_without_answer = "\n".join(question_lines).strip()

    # 儲存最新題目供 webhook 檢查
    with open("latest_question.txt", "w", encoding="utf-8") as f:
        f.write(f"{correct_answer}|{question_without_answer}")

    # 發送給學生
    message = PushMessageRequest(
        to=user_id,
        messages=[
            TextMessage(
                text=f"🆕 題目練習：\n{question_without_answer}\n\n請回覆 A / B / C / D 作答，我會告訴你對不對喔！"
            )
        ]
    )
    line_bot_api.push_message(message)
    print("✅ 已送出 GPT 題目到學生 LINE！")

else:
    print("🎉 沒有錯題了！太棒了！")


📄 處理檔案：student.docx
👉 答案數量：學生 10，標準 10
🎯 student.docx 總分：20 / 100
📄 處理檔案：student1.docx
👉 答案數量：學生 10，標準 10
🎯 student1.docx 總分：10 / 100
✅ 已送出 GPT 題目到學生 LINE！


In [7]:
!pip install python-dotenv